# Dinoflagellate and Diatom in a Storm Event

In this notebook, we will first generate section plots which show the biological response in 3 typical regions in SoG, and then the time series of 2 planktons and their ratios in 3 regions during the storm.

## Section Plot

In [6]:
# Bio_Section_Plot.py
# 目标：绘制硅藻 (Diatoms) 和 鞭毛藻 (Flagellates) 在东西向断面上的深度分布图。
# 色标：使用感知均匀且高对比度的 viridis。
import netCDF4 as nc
import numpy as np
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe  # 用于给文字加描边，防止看不清
from matplotlib.colors import LogNorm  # 用于对数色标

# --- 1. 配置与路径设置 ---
BASE_DIR = '/results2/SalishSea/nowcast-green.201905/'
FNAME_HEAD = 'SalishSea_1d_' 
FNAME_TAIL = '_ptrc_T.nc'
NEW_INDIR_RESULTS = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Section_Biology/'

if not os.path.exists(NEW_INDIR_RESULTS):
    os.makedirs(NEW_INDIR_RESULTS)

# 目标点定义
TARGETS_DEFINITIONS = {
    'Point_1': {'value': (49.0, -123.25), 'label': 'Section#1'},
    'Point_2': {'value': (49.3, -124.0),  'label': 'Section#2'},
    'Point_3': {'value': (49.9, -124.8),  'label': 'Section#3'},
}

# 断面宽度配置 (基于索引偏移)
SECTION_CONFIG = {
    'Point_1': {'left_offset': 35, 'right_offset': 22},
    'Point_2': {'left_offset': 30, 'right_offset': 70},
    'Point_3': {'left_offset': 45, 'right_offset': 25},
}

# 绘图设置
year_target = [2011, 2018]
month_target = 7
target_days = [8, 17]
DEPTH_LIMIT = 50

# --- 2. 辅助函数 ---

def find_nearest_indices_2d(lon_2d, lat_2d, target_lon, target_lat):
    dist_sq = (lon_2d - target_lon)**2 + (lat_2d - target_lat)**2
    return np.unravel_index(dist_sq.argmin(), lon_2d.shape)

def load_bio_data_refined(date_obj, y_idx, x_idx, config, lon_2d):
    """根据偏移量切出精确的断面范围"""
    file_path = os.path.join(BASE_DIR, date_obj.strftime('%d%b%y').lower(), 
                             f"{FNAME_HEAD}{date_obj.strftime('%Y%m%d')}_{date_obj.strftime('%Y%m%d')}{FNAME_TAIL}")
    
    if not os.path.exists(file_path): return None
    
    with nc.Dataset(file_path, 'r') as ncfile:
        z_var = ncfile.variables['deptht'][:]
        z_idx_limit = np.abs(z_var - DEPTH_LIMIT).argmin()
        
        # 确定 X 轴切片范围
        x_start = max(0, x_idx - config['left_offset'])
        x_end = min(lon_2d.shape[1], x_idx + config['right_offset'])
        x_slice = slice(x_start, x_end)
        
        diat = ncfile.variables['diatoms'][0, :z_idx_limit+1, y_idx, x_slice]
        flag = ncfile.variables['flagellates'][0, :z_idx_limit+1, y_idx, x_slice]
        lon_sec = lon_2d[y_idx, x_slice]
        
        return diat, flag, lon_sec, z_var[:z_idx_limit+1]

def plot_refined_section(data, lon, depths, date_str, year, label, var_type):
    """
    修改点：
    1. 使用 np.ma.masked_where 处理 0 值。
    2. 设置 ax.set_facecolor 为灰色（陆地颜色）。
    3. 去掉 set_title，改用 ax.text 在图内添加日期标签。
    """
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # --- 修改1：设置背景色（即陆地/无效值的颜色）---
    # 'darkgray' 或者 '#bfbfbf' 都是比较标准的陆地填充色
    ax.set_facecolor('darkgray') 
    
    # 针对不同生物设置色标和范围
    if var_type == 'Diatom':
        cmap = 'YlGnBu' 
        vmin, vmax = 0.1, 15  # 稍微调整一下范围适应 log
    else:
        cmap = 'RdPu'   
        vmin, vmax = 0.05, 5

    # --- 修改2：数据掩膜 (Masking) ---
    # 将 <= 0 的数据掩盖掉（变为无效值），这样 pcolormesh 就不会绘制这些点
    # 从而露出底下的灰色背景
    data_masked = np.ma.masked_less_equal(data, 0)

    X, Z = np.meshgrid(lon, depths)
    
    # 绘图 (注意使用 masked 数据)
    cf = ax.pcolormesh(X, Z, data_masked, 
                       norm=LogNorm(vmin=vmin, vmax=vmax), 
                       cmap=cmap, shading='auto')
    
    ax.invert_yaxis()
    ax.set_ylim(depths.max(), 0) # 确保Y轴也是从0开始向下
    
    # 坐标轴标签
    ax.set_ylabel('Depth (m)', fontsize=11)
    ax.set_xlabel('Longitude ($^{\circ}$E)', fontsize=11)
    
    # --- 修改3：移除标题，添加图内文字 ---
    # transform=ax.transAxes 表示使用相对坐标 (0-1)
    # (0.98, 0.03) 表示右下角; ha='right', va='bottom' 对齐方式
    text_content = f"{label}\n{date_str}"
    
    t = ax.text(0.97, 0.05, text_content, 
                transform=ax.transAxes, 
                ha='right', va='bottom', 
                fontsize=14, fontweight='bold', color='white')
    
    # 给文字加个黑色描边，防止背景太浅看不清
    t.set_path_effects([pe.withStroke(linewidth=2, foreground='black')])

    # 色标
    cbar = fig.colorbar(cf, ax=ax, extend='both', pad=0.02)
    cbar.set_label(f'{var_type} ($\mu$mol N L$^{{-1}}$)', fontsize=11)
    
    # 保存
    out_name = f"Refined_{var_type}_{year}_{date_str.replace('-','')}_{label.replace('#','')}.svg"
    plt.savefig(os.path.join(NEW_INDIR_RESULTS, out_name), dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved: {out_name}")

# --- 3. 主程序 ---

def main():
    # 预加载坐标
    with nc.Dataset(os.path.join(BASE_DIR, '08jul11/SalishSea_1d_20110708_20110708_ptrc_T.nc'), 'r') as nf:
        lon_2d = nf.variables['nav_lon'][:]
        lat_2d = nf.variables['nav_lat'][:]

    for year in year_target:
        for day in target_days:
            curr_date = datetime.date(year, month_target, day)
            date_s = curr_date.strftime('%Y-%m-%d')
            
            for p_id, p_def in TARGETS_DEFINITIONS.items():
                # 寻找中心点索引
                y_idx, x_idx = find_nearest_indices_2d(lon_2d, lat_2d, p_def['value'][1], p_def['value'][0])
                
                # 加载切片数据
                res = load_bio_data_refined(curr_date, y_idx, x_idx, SECTION_CONFIG[p_id], lon_2d)
                if res is None: continue
                
                diat, flag, lon_s, depth_s = res
                
                # 绘图
                plot_refined_section(diat, lon_s, depth_s, date_s, year, p_def['label'], 'Diatom')
                plot_refined_section(flag, lon_s, depth_s, date_s, year, p_def['label'], 'Flagellate')

    print(f"Done! Check results in {NEW_INDIR_RESULTS}")

if __name__ == '__main__':
    main()

<>:107: SyntaxWarning: invalid escape sequence '\c'
<>:124: SyntaxWarning: invalid escape sequence '\m'
<>:107: SyntaxWarning: invalid escape sequence '\c'
<>:124: SyntaxWarning: invalid escape sequence '\m'
/tmp/ipykernel_2830240/1217458563.py:107: SyntaxWarning: invalid escape sequence '\c'
  ax.set_xlabel('Longitude ($^{\circ}$E)', fontsize=11)
/tmp/ipykernel_2830240/1217458563.py:124: SyntaxWarning: invalid escape sequence '\m'
  cbar.set_label(f'{var_type} ($\mu$mol N L$^{{-1}}$)', fontsize=11)


Saved: Refined_Diatom_2011_20110708_Section1.svg
Saved: Refined_Flagellate_2011_20110708_Section1.svg
Saved: Refined_Diatom_2011_20110708_Section2.svg
Saved: Refined_Flagellate_2011_20110708_Section2.svg
Saved: Refined_Diatom_2011_20110708_Section3.svg
Saved: Refined_Flagellate_2011_20110708_Section3.svg
Saved: Refined_Diatom_2011_20110717_Section1.svg
Saved: Refined_Flagellate_2011_20110717_Section1.svg
Saved: Refined_Diatom_2011_20110717_Section2.svg
Saved: Refined_Flagellate_2011_20110717_Section2.svg
Saved: Refined_Diatom_2011_20110717_Section3.svg
Saved: Refined_Flagellate_2011_20110717_Section3.svg
Saved: Refined_Diatom_2018_20180708_Section1.svg
Saved: Refined_Flagellate_2018_20180708_Section1.svg
Saved: Refined_Diatom_2018_20180708_Section2.svg
Saved: Refined_Flagellate_2018_20180708_Section2.svg
Saved: Refined_Diatom_2018_20180708_Section3.svg
Saved: Refined_Flagellate_2018_20180708_Section3.svg
Saved: Refined_Diatom_2018_20180717_Section1.svg
Saved: Refined_Flagellate_2018_20

## Region Time Series Plot

In [10]:
# Data Extract and Integrate

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import netCDF4 as nc
import numpy as np
import datetime
import os
import pickle
from tqdm import tqdm

# ================= 配置与路径 =================
# 1. 基础路径 (请确保这些路径在服务器上存在)
BASE_DIR = '/results2/SalishSea/nowcast-green.201905/'
FILE_AREA = '/results2/SalishSea/nowcast-green.201905/01apr07/SalishSea_1h_20070401_20070401_ptrc_T.nc'
FILE_E3T = '/results2/SalishSea/month-avg.201905/SalishSeaCast_1m_carp_T_20070101_20070131.nc'
OUTPUT_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/'
SAVE_FILENAME = 'extracted_biomass_data.pkl'  # 文件名改一下，避免覆盖之前的

# 2. 变量名
VAR_DIATOMS = 'diatoms'
VAR_FLAGELLATES = 'flagellates'
DEPTH_LIMIT = 20.0 

# 3. 定义目标点
TARGETS_DEFINITIONS = {
    'Point_1': {'value': (49.0, -123.25), 'label': 'Point#1'},
    'Point_2': {'value': (49.3, -124.0),  'label': 'Point#2'},
    'Point_3': {'value': (49.9, -124.8),  'label': 'Point#3'},
}

# 4. 定义区域大小
# 半径 10 = 直径 21 个网格点
GRID_BOX_RADIUS = 10  

# ================= 核心工具函数 =================

def get_indices_and_weights(file_area, file_e3t, lat_t, lon_t, radius, depth_limit):
    """
    根据经纬度找到网格索引，并计算该区域前20米的体积权重
    """
    with nc.Dataset(file_area, 'r') as nca, nc.Dataset(file_e3t, 'r') as nce:
        lats = nca.variables['nav_lat'][:]
        lons = nca.variables['nav_lon'][:]
        depths = nce.variables['depth'][:]
        
        # 1. 深度截断 (找到第一个大于20m的层的索引)
        # 注意：SalishSeaCast 模型深度通常固定，这里取第一个大于 limit 的层索引
        k_indices = np.where(depths > depth_limit)[0]
        k_limit = k_indices[0] if len(k_indices) > 0 else len(depths)
        
        # 2. 空间搜索 (找最近的点)
        dist_sq = (lons - lon_t)**2 + (lats - lat_t)**2
        j_c, i_c = np.unravel_index(np.argmin(dist_sq), dist_sq.shape)
        
        # 3. 确定切片范围 (注意边界处理)
        j1 = max(0, j_c - radius)
        j2 = j_c + radius + 1
        i1 = max(0, i_c - radius)
        i2 = i_c + radius + 1
        
        # 4. 读取该小块的 Area 和 E3T (层厚)
        # area shape: (j_sub, i_sub)
        area_sub = nca.variables['area'][j1:j2, i1:i2]
        # e3t shape: (1, k_limit, j_sub, i_sub) -> 取[0]变成 (k, j, i)
        e3t_sub = nce.variables['e3t'][0, :k_limit, j1:j2, i1:i2]
        
        # 5. 计算体积 (k, j, i)
        # 利用广播机制: (j, i) * (k, j, i) -> (k, j, i)
        volume_sub = area_sub * e3t_sub
        
        return j1, j2, i1, i2, k_limit, volume_sub

# ================= 主逻辑 =================

if __name__ == "__main__":
    # 确保输出目录存在
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    all_results = {}

    for key, info in TARGETS_DEFINITIONS.items():
        print(f"\n>>>> Processing Target: {info['label']} {info['value']} <<<<")
        
        lat, lon = info['value']
        
        # 1. 预计算网格和权重 (只做一次)
        try:
            j1, j2, i1, i2, k_max, vol_weight = get_indices_and_weights(
                FILE_AREA, FILE_E3T, lat, lon, GRID_BOX_RADIUS, DEPTH_LIMIT
            )
            print(f"  Grid slice: y[{j1}:{j2}], x[{i1}:{i2}], depth[0:{k_max}]")
        except Exception as e:
            print(f"  Error locating grid: {e}")
            continue

        all_results[key] = {
            'meta': info,
            'data': {}
        }

        for year in [2011, 2018]:
            print(f"  -- Year {year} --")
            
            # >>>>> 修改点在这里：日期范围改成 6月1日 到 9月30日 <<<<<
            start_date = datetime.datetime(year, 6, 1)
            end_date = datetime.datetime(year, 9, 30) 
            
            total_days = (end_date - start_date).days + 1
            
            # 初始化存储列表
            times_list = []
            d_list = []
            f_list = []
            ratio_list = []
            
            pbar = tqdm(total=total_days, desc=f"    Extracting", unit="day")
            current_date = start_date
            
            while current_date <= end_date:
                # 构造文件路径
                date_str = current_date.strftime('%Y%m%d')
                folder_date = current_date.strftime('%d%b%y').lower()
                fname = f"SalishSea_1h_{date_str}_{date_str}_ptrc_T.nc"
                file_path = os.path.join(BASE_DIR, folder_date, fname)
                
                if os.path.exists(file_path):
                    try:
                        with nc.Dataset(file_path, 'r') as ds:
                            # 提取浓度数据 (time, k, j, i)
                            # 假设文件里是每小时数据 (24个时间步)
                            conc_d = ds.variables[VAR_DIATOMS][:, :k_max, j1:j2, i1:i2]
                            conc_f = ds.variables[VAR_FLAGELLATES][:, :k_max, j1:j2, i1:i2]
                            
                            # 计算总生物量 (Integral) = sum(Conc * Vol)
                            # 结果维度变成 (24,)，即每小时的一个总值
                            inv_d_hourly = np.sum(conc_d * vol_weight, axis=(1, 2, 3))
                            inv_f_hourly = np.sum(conc_f * vol_weight, axis=(1, 2, 3))
                            
                            # 降采样：每6小时存一个点
                            for h in range(0, 24, 6):
                                # 取6小时平均
                                val_d = np.mean(inv_d_hourly[h:h+6])
                                val_f = np.mean(inv_f_hourly[h:h+6])
                                
                                # 计算比值 (防除零)
                                val_r = val_d / val_f if val_f > 1e-10 else 0.0
                                
                                # 记录时间 (使用当前日期 + 小时)
                                t_point = current_date + datetime.timedelta(hours=h)
                                
                                times_list.append(t_point)
                                d_list.append(val_d)
                                f_list.append(val_f)
                                ratio_list.append(val_r)
                                
                    except Exception as e:
                        pbar.write(f"    Error processing {date_str}: {e}")
                
                current_date += datetime.timedelta(days=1)
                pbar.update(1)
            
            pbar.close()
            
            # 存入字典
            all_results[key]['data'][year] = {
                'times': times_list,
                'diatoms': d_list,
                'flagellates': f_list,
                'ratio': ratio_list
            }

    # 保存结果到 PKL
    out_pkl = os.path.join(OUTPUT_DIR, SAVE_FILENAME)
    print(f"\nWriting results to: {out_pkl}")
    with open(out_pkl, 'wb') as f:
        pickle.dump(all_results, f)
    print("Done. Now you can run step2_plot_data.py")


>>>> Processing Target: Point#1 (49.0, -123.25) <<<<
  Grid slice: y[393:414], x[277:298], depth[0:19]
  -- Year 2011 --


    Extracting: 100%|██████████| 122/122 [22:59<00:00, 11.31s/day]


  -- Year 2018 --


    Extracting: 100%|██████████| 122/122 [24:19<00:00, 11.97s/day]



>>>> Processing Target: Point#2 (49.3, -124.0) <<<<
  Grid slice: y[505:526], x[205:226], depth[0:19]
  -- Year 2011 --


...


>>>> Processing Target: Point#3 (49.9, -124.8) <<<<
  Grid slice: y[679:700], x[162:183], depth[0:19]
  -- Year 2011 --


    Extracting: 100%|██████████| 122/122 [19:22<00:00,  9.53s/day]


  -- Year 2018 --


    Extracting: 100%|██████████| 122/122 [19:20<00:00,  9.51s/day]


Writing results to: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/extracted_biomass_data.pkl
Done. Now you can run step2_plot_data.py


In [ ]:
# Plot 

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pickle
import os
import datetime

# ================= 配置 =================
INPUT_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/'
DATA_FILENAME = 'extracted_biomass_data.pkl'

# 2. 定义关键时间节点
BASE_YEAR = 2011
STORM_START_MD = (7, 9)   # 风暴开始
STORM_END_MD = (7, 17)    # 风暴结束 (持续时间较长)

# 3. 绘图聚焦窗口 (X轴范围)
# 稍微缩窄一点范围，聚焦在风暴前后
PLOT_X_LIM_START = datetime.datetime(BASE_YEAR, 7, 5)
PLOT_X_LIM_END = datetime.datetime(BASE_YEAR, 7, 22)

# ================= 绘图核心函数 =================
def plot_storm_focus_clean(region_key, region_data, save_dir):
    label = region_data['meta']['label']
    years_data = region_data['data']
    
    # 创建 3行1列 的画布
    fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    
    # 颜色定义
    color_2011 = 'tab:blue'
    color_2018 = 'tab:red'
    # 修改：风暴区域颜色 (淡钢蓝，更有风暴的感觉，且不抢眼)
    color_storm_area = 'lightsteelblue' 
    
    vars_config = [
        ('diatoms', 'Diatoms Inventory (mmol N)', axes[0]),
        ('flagellates', 'Flagellates Inventory (mmol N)', axes[1]),
        ('ratio', 'Ratio (Diatoms / Flagellates)', axes[2])
    ]
    
    # --- 1. 绘制数据曲线 ---
    for year in [2011, 2018]:
        if year not in years_data: continue
        yd = years_data[year]
        
        # 对齐时间轴到 2011 年
        times = yd['times']
        fake_times = [t.replace(year=BASE_YEAR) for t in times]
        
        # 绘图颜色
        c = color_2011 if year == 2011 else color_2018
        
        for var_name, title, ax in vars_config:
            vals = yd[var_name]
            # 线条稍微加粗
            ax.plot(fake_times, vals, label=f'{year}', color=c, linewidth=2, alpha=0.85)

    # --- 2. 添加风暴区域 (遍历所有子图) ---
    t_storm_start = datetime.datetime(BASE_YEAR, *STORM_START_MD)
    t_storm_end = datetime.datetime(BASE_YEAR, *STORM_END_MD)
    
    for i, (var_name, title, ax) in enumerate(vars_config):
        # A. 绘制风暴区 (无边框)
        ax.axvspan(t_storm_start, t_storm_end, color=color_storm_area, alpha=0.4, zorder=0)
        
        # B. 基础设置
        ax.set_ylabel(title, fontsize=10, fontweight='bold')
        ax.grid(True, linestyle=':', alpha=0.6)
        
        # 图例只在第一张图显示
        if i == 0:
            ax.legend(loc='upper right', frameon=True, framealpha=0.9)

        # C. 仅在最上面的图添加 "Storm Event" 文字
        if i == 0:
            y_lims = ax.get_ylim()
            text_y = y_lims[1] - (y_lims[1]-y_lims[0])*0.05
            # 文字居中于风暴区
            mid_storm = t_storm_start + (t_storm_end - t_storm_start)/2
            
            ax.text(mid_storm, text_y, 'Storm Event\n(Mixing)', 
                    ha='center', va='top', color='steelblue', 
                    fontweight='bold', fontsize=10,
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

    # --- 3. 定制 X 轴 (只操作最下面的图) ---
    bottom_ax = axes[2]
    
    # A. 设置范围
    bottom_ax.set_xlim(PLOT_X_LIM_START, PLOT_X_LIM_END)
    
    # B. 设置特定刻度 (只显示风暴开始和结束)
    target_ticks = [t_storm_start, t_storm_end]
    bottom_ax.set_xticks(target_ticks)
    
    # C. 日期格式
    bottom_ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    
    # D. 给刻度标签上色
    fig.canvas.draw() 
    xtick_labels = bottom_ax.get_xticklabels()
    
    for label_obj in xtick_labels:
        txt = label_obj.get_text()
        # 只要是风暴相关的日期，都标为风暴色
        label_obj.set_color('steelblue')
        label_obj.set_fontweight('bold')
            
    # 加重风暴起止的垂直网格线
    bottom_ax.grid(True, which='major', axis='x', linestyle='--', linewidth=1.5, color=color_storm_area)

    # --- 4. 保存 ---
    fig.suptitle(f'Biomass Response to Storm: {label}', fontsize=14, y=0.98)
    plt.tight_layout()
    plt.subplots_adjust(top=0.92, hspace=0.1) 
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    out_name = f'StormFocus_Clean_{region_key}.png' # 文件名加了 _Clean
    save_path = os.path.join(save_dir, out_name)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved plot: {save_path}")


# ================= 主程序 =================
if __name__ == "__main__":
    pkl_path = os.path.join(INPUT_DIR, DATA_FILENAME)
    
    if not os.path.exists(pkl_path):
        print(f"Error: Data file not found: {pkl_path}")
        exit()
        
    print(f"Loading data from {DATA_FILENAME}...")
    with open(pkl_path, 'rb') as f:
        all_results = pickle.load(f)
        
    print("Generating clean focused plots...")
    for key, data in all_results.items():
        plot_storm_focus_clean(key, data, OUTPUT_DIR)
        
    print("Done.")

Loading data...
Plotting...
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/TimeSeries_3Panel_Point_1.png
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/TimeSeries_3Panel_Point_2.png
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/TimeSeries_3Panel_Point_3.png
All plots generated.


In [5]:
# During Storm Plot

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pickle
import os
import datetime
import numpy as np

# ================= 配置 =================
# 1. 路径与文件
INPUT_DIR = '/home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/'
DATA_FILENAME = 'extracted_biomass_data.pkl'
OUTPUT_DIR = os.path.join(INPUT_DIR, 'Storm_Focus_Plots') 

# 2. 定义关键时间节点
BASE_YEAR = 2011
STORM_START_MD = (7, 9)   # 风暴开始
STORM_END_MD = (7, 17)    # 风暴结束 (持续时间较长)

# 3. 绘图聚焦窗口 (X轴范围)
# 稍微缩窄一点范围，聚焦在风暴前后
PLOT_X_LIM_START = datetime.datetime(BASE_YEAR, 7, 5)
PLOT_X_LIM_END = datetime.datetime(BASE_YEAR, 7, 22)

# ================= 绘图核心函数 =================
def plot_storm_focus_clean(region_key, region_data, save_dir):
    label = region_data['meta']['label']
    years_data = region_data['data']
    
    # 创建 3行1列 的画布
    fig, axes = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
    
    # 颜色定义
    color_2011 = 'tab:blue'
    color_2018 = 'tab:red'
    # 修改：风暴区域颜色 (淡钢蓝，更有风暴的感觉，且不抢眼)
    color_storm_area = 'lightsteelblue' 
    
    vars_config = [
        ('diatoms', 'Diatoms Inventory (mmol N)', axes[0]),
        ('flagellates', 'Flagellates Inventory (mmol N)', axes[1]),
        ('ratio', 'Ratio (Diatoms / Flagellates)', axes[2])
    ]
    
    # --- 1. 绘制数据曲线 ---
    for year in [2011, 2018]:
        if year not in years_data: continue
        yd = years_data[year]
        
        # 对齐时间轴到 2011 年
        times = yd['times']
        fake_times = [t.replace(year=BASE_YEAR) for t in times]
        
        # 绘图颜色
        c = color_2011 if year == 2011 else color_2018
        
        for var_name, title, ax in vars_config:
            vals = yd[var_name]
            # 线条稍微加粗
            ax.plot(fake_times, vals, label=f'{year}', color=c, linewidth=2, alpha=0.85)

    # --- 2. 添加风暴区域 (遍历所有子图) ---
    t_storm_start = datetime.datetime(BASE_YEAR, *STORM_START_MD)
    t_storm_end = datetime.datetime(BASE_YEAR, *STORM_END_MD)
    
    for i, (var_name, title, ax) in enumerate(vars_config):
        # A. 绘制风暴区 (无边框)
        ax.axvspan(t_storm_start, t_storm_end, color=color_storm_area, alpha=0.4, zorder=0)
        
        # B. 基础设置
        ax.set_ylabel(title, fontsize=10, fontweight='bold')
        ax.grid(True, linestyle=':', alpha=0.6)
        
        # 图例只在第一张图显示
        if i == 0:
            ax.legend(loc='upper right', frameon=True, framealpha=0.9)

        # C. 仅在最上面的图添加 "Storm Event" 文字
        if i == 0:
            y_lims = ax.get_ylim()
            text_y = y_lims[1] - (y_lims[1]-y_lims[0])*0.05
            # 文字居中于风暴区
            mid_storm = t_storm_start + (t_storm_end - t_storm_start)/2
            
            ax.text(mid_storm, text_y, 'Storm Event', 
                    ha='center', va='top', color='steelblue', 
                    fontweight='bold', fontsize=10,
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

    # --- 3. 定制 X 轴 (只操作最下面的图) ---
    bottom_ax = axes[2]
    
    # A. 设置范围
    bottom_ax.set_xlim(PLOT_X_LIM_START, PLOT_X_LIM_END)
    
    # B. 设置特定刻度 (只显示风暴开始和结束)
    target_ticks = [t_storm_start, t_storm_end]
    bottom_ax.set_xticks(target_ticks)
    
    # C. 日期格式
    bottom_ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    
    # D. 给刻度标签上色
    fig.canvas.draw() 
    xtick_labels = bottom_ax.get_xticklabels()
    
    for label_obj in xtick_labels:
        txt = label_obj.get_text()
        # 只要是风暴相关的日期，都标为风暴色
        label_obj.set_color('steelblue')
        label_obj.set_fontweight('bold')
            
    # 加重风暴起止的垂直网格线
    bottom_ax.grid(True, which='major', axis='x', linestyle='--', linewidth=1.5, color=color_storm_area)

    # --- 4. 保存 ---
    fig.suptitle(f'Biomass Response to Storm: {label}', fontsize=14, y=0.98)
    plt.tight_layout()
    plt.subplots_adjust(top=0.92, hspace=0.1) 
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    out_name = f'StormFocus_Clean_{region_key}.svg' # 文件名加了 _Clean
    save_path = os.path.join(save_dir, out_name)
    plt.savefig(save_path, dpi=300)
    plt.close()
    print(f"Saved plot: {save_path}")


# ================= 主程序 =================
if __name__ == "__main__":
    pkl_path = os.path.join(INPUT_DIR, DATA_FILENAME)
    
    if not os.path.exists(pkl_path):
        print(f"Error: Data file not found: {pkl_path}")
        exit()
        
    print(f"Loading data from {DATA_FILENAME}...")
    with open(pkl_path, 'rb') as f:
        all_results = pickle.load(f)
        
    print("Generating clean focused plots...")
    for key, data in all_results.items():
        plot_storm_focus_clean(key, data, OUTPUT_DIR)
        
    print("Done.")

Loading data from extracted_biomass_data.pkl...
Generating clean focused plots...
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/Storm_Focus_Plots/StormFocus_Clean_Point_1.svg
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/Storm_Focus_Plots/StormFocus_Clean_Point_2.svg
Saved plot: /home/jqiu/Programing/Projects/analysis-junqi/Diatom_vs_Flagellate_Report/Results_Bio/Storm_Focus_Plots/StormFocus_Clean_Point_3.svg
Done.
